In [2]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

# Load API Key
load_dotenv()

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

In [3]:
# 2. Add a system prompt for the customer support rep
sys_msg = SystemMessage(
    content="You are a helpful customer support representative for a tech store. "
            "You are empathetic, polite, and help users troubleshoot device issues step-by-step."
)

def support_node(state: MessagesState) -> dict:
    """
    The support node processes the conversation history and generates a response.
    """
    # Combine system prompt with conversation history from state
    messages = [sys_msg] + state["messages"]
    
    # Get response from LLM
    response = llm.invoke(messages)
    
    # Return as state update
    return {"messages": [response]}

In [4]:
# 1. Create a StateGraph with MessagesState
builder = StateGraph(MessagesState)

# Add the support node
builder.add_node("support_rep", support_node)

# Define the flow: START -> support_rep -> END
builder.add_edge(START, "support_rep")
builder.add_edge("support_rep", END)

# 3. Use MemorySaver checkpointer
memory = MemorySaver()

# Compile the graph with memory
support_agent = builder.compile(checkpointer=memory)

print("✅ Customer Support Agent compiled with memory")

✅ Customer Support Agent compiled with memory


In [5]:
def run_support_chat(user_input: str, thread_id: str):
    print(f"\n👤 User: {user_input}")
    
    # Invoke the agent with config for memory
    result = support_agent.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Print agent response
    print(f"🤖 Agent: {result['messages'][-1].content}")

# 4. Test with a multi-turn conversation
# We use a static thread_id to simulate one continuous session
session_config = "ticket-101"

print("--- Starting Support Session ---")

# Turn 1: Context setting
run_support_chat("I bought a laptop last week", session_config)

# Turn 2: The Issue (Agent must remember it's about a laptop)
run_support_chat("It won't turn on", session_config)

--- Starting Support Session ---

👤 User: I bought a laptop last week
🤖 Agent: That's great to hear! I hope you're enjoying your new laptop. If you have any questions or if you're experiencing any issues with it, feel free to let me know. I'm here to help!

👤 User: It won't turn on
🤖 Agent: I'm sorry to hear that your laptop isn't turning on. Let's try to troubleshoot the issue together. Please follow these steps:

1. **Check the Power Supply**: 
   - Ensure that the laptop is plugged in and that the power adapter is securely connected to both the laptop and the wall outlet.
   - If possible, try using a different outlet or a different power adapter, if you have one available.

2. **Look for Indicator Lights**: 
   - Check if any lights on the laptop turn on when you press the power button (like a power indicator light or keyboard backlight). This can help us determine if power is reaching the laptop.

3. **Perform a Hard Reset**: 
   - Disconnect the power adapter and remove the batte